In [1]:
import sys
sys.path.append('../')

In [2]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model


In [3]:
import xarray
import numpy
import matplotlib.pyplot as plt

In [4]:
deckSet=['piControl','historical']
scenarioSet=['ssp126', 'ssp245', 'ssp370','ssp585']
experimentSet=[*deckSet, *scenarioSet]

In [5]:
modelSet=_model.scenarioMip

In [7]:
for i in range(0,len(modelSet)):
  # everyone loves nested for loops  
    iModel=modelSet[i]
    print(str(i) + str(iModel))
    
    for experiment in experimentSet: 
        
        if experiment=='piControl':
            variant = iModel[2]
        else:
            variant = iModel[3]

        try:
            indecesDs = xarray.open_dataset('../results/cmipWarmSeasonIndeces/' + iModel[1] +'_'+ experiment + '.nc')

            calc=[
                indecesDs.year.values[0], 
                indecesDs.year.values[-1]+1
            ]
            

            for variable in ['ts_Amon', 'psl_Amon']:#, 'pr_Amon', 'tas_Amon']:
                
                sourceXr = fh.loadModelData(iModel[1], variable, experiment, variant)
                
                if experiment=='piControl':
                    source=[sourceXr.time.dt.year.values[0],
                        sourceXr.time.dt.year.values[-1]]
                elif experiment=='historical':
                    histStart=sourceXr.time.dt.year.values[0]
                    source=[histStart,
                        sourceXr.time.dt.year.values[-1]]
                else:
                    source=[histStart,
                        sourceXr.time.dt.year.values[-1]]
                
                if not(numpy.array_equal(source,calc)):
                    print('noMatch' + experiment + variable)
                    print([*calc, *source])
                if all(
                [calc[1]-calc[0]<498, 
                 experiment=='piControl']
                ):
                    print('piControl too short')
                    print([*calc, *source])
        
        
        except Exception as e:
            print(e)
        
        

0['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1' 'r1i1p1f1']
1['CSIRO' 'ACCESS-ESM1-5' 'r1i1p1f1' 'r1i1p1f1']
2['AWI' 'AWI-CM-1-1-MR' 'r1i1p1f1' 'r1i1p1f1']
3['BCC' 'BCC-CSM2-MR' 'r1i1p1f1' 'r1i1p1f1']
4['CAMS' 'CAMS-CSM1-0' 'r1i1p1f1' 'r1i1p1f1']
5['CAS' 'CAS-ESM2-0' 'r1i1p1f1' 'r1i1p1f1']
6['NCAR' 'CESM2' 'r1i1p1f1' 'r10i1p1f1']


/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packa

7['NCAR' 'CESM2-WACCM' 'r1i1p1f1' 'r1i1p1f1']


/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/home/steketea/anaconda3/envs/testEnv/lib/python3.9/site-packa

8['THU' 'CIESM' 'r1i1p1f1' 'r1i1p1f1']
[Errno 2] No such file or directory: b'/home/steketea/compound/results/cmipWarmSeasonIndeces/CIESM_ssp370.nc'
9['CMCC' 'CMCC-CM2-SR5' 'r1i1p1f1' 'r1i1p1f1']
10['CMCC' 'CMCC-ESM2' 'r1i1p1f1' 'r1i1p1f1']
11['CNRM-CERFACS' 'CNRM-CM6-1' 'r1i1p1f2' 'r1i1p1f2']
12['CNRM-CERFACS' 'CNRM-CM6-1-HR' 'r1i1p1f2' 'r1i1p1f2']
piControl too short
[Errno 2] No such file or directory: b'/home/steketea/compound/results/cmipWarmSeasonIndeces/CNRM-CM6-1-HR_ssp370.nc'
13['CNRM-CERFACS' 'CNRM-ESM2-1' 'r1i1p1f2' 'r1i1p1f2']
14['CCCma' 'CanESM5' 'r1i1p1f1' 'r1i1p1f1']
15['CCCma' 'CanESM5-CanOE' 'r1i1p2f1' 'r1i1p2f1']
16['EC-Earth-Consortium' 'EC-Earth3' 'r1i1p1f1' 'r1i1p1f1']
17['EC-Earth-Consortium' 'EC-Earth3-CC' 'r1i1p1f1' 'r1i1p1f1']
[Errno 2] No such file or directory: b'/home/steketea/compound/results/cmipWarmSeasonIndeces/EC-Earth3-CC_ssp126.nc'
[Errno 2] No such file or directory: b'/home/steketea/compound/results/cmipWarmSeasonIndeces/EC-Earth3-CC_ssp370.nc'
18['